In [1]:
import jax
import jax.numpy as jnp
from jax import jit, vmap, grad
from jax import random
import optax
import numpy as np
import pickle
from scipy.stats import gamma as gamma_dist

from jax.scipy.special import gamma
TN = 1000

In [2]:
def init_network_params(sizes, key=random.PRNGKey(4)):
    def random_layer_params(m, n, key, scale=1e-2):
        w_key, b_key = random.split(key)
        return scale * random.uniform(w_key, (n, m), minval=1, maxval=3), scale * random.uniform(b_key, (n,), minval=1, maxval=3)

    keys = random.split(key, len(sizes))
    return [random_layer_params(m, n, k) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

def predict_(NN, u):
    activations = u.reshape((-1, ))
    activations = jnp.log(activations/(1-activations))
    for w, b in NN[:-1]:
        outputs = jnp.dot(w, activations) + b
        activations = jax.nn.sigmoid(outputs)
    final_w, final_b = NN[-1]
    logit = jnp.dot(final_w, activations) + final_b
    return logit.squeeze()

def predict(NN, u):
    return predict_(NN, u) + gamma_dist.ppf(0.5, 2.5) - predict_(NN, np.array([0.5]))

vpredict = vmap(predict, in_axes=(None, 0))
gpredict = grad(predict, argnums=1)
vgpredict = jit(vmap(gpredict, in_axes=(None, 0)))

In [3]:
def pi(x):
    k = 2.5   # shape parameter (non-integer)
    theta = 1.0  # scale parameter
    coef = (x ** (k - 1)) * jnp.exp(-x / theta) / (gamma(k) * (theta ** k))
    return coef
vpi = vmap(pi, in_axes=(0,))

@jit
def obj(NN, test_uz):
    def ell(NN, uz):
        u, z = uz
        return jnp.square(gpredict(NN, u)-1/pi(z))
    vell = vmap(ell, in_axes=(None, 0))

    result = vell(NN, test_uz)
    result = jnp.mean(result)
    return result

@jit
def constraint(NN, test_uz):
    u_batch, z_batch = test_uz[:, 0], test_uz[:, 1]
    epsilon = 1e-12
    def g(NN, u):
        return jnp.maximum(-gpredict(NN, u)+epsilon, 0)
    vg = vmap(g, in_axes=(None, 0))

    result = vg(NN, u_batch)
    result = jnp.sum(result)
    return result

gobj = jit(grad(obj))
gconst = jit(grad(constraint))

@jit
def evaluation(NN, test_uz):
    u_batch, z_batch = test_uz[:, 0], test_uz[:, 1]
    result = jnp.square(vpredict(NN, u_batch) - z_batch)
    result = jnp.mean(result)
    return result

In [4]:
LAYER_SIZES = [1, 10, 10, 10, 1]
NN = init_network_params(LAYER_SIZES, random.PRNGKey(15))

optimizer= optax.adam(0.001)
opt_state = optimizer.init(NN)
bug = 0
co = 0
test_uz = np.load(f'/content/drive/MyDrive/No. 25 Inverse Problem/Gamma (PINN)/test_uz.npy')
train_uz = np.load(f'/content/drive/MyDrive/No. 25 Inverse Problem/Gamma (PINN)/train_uz.npy')[:TN]
lowest = evaluation(NN, test_uz)
Lloss = []
Llowest = []
NN_best = NN
for epoch in range(2000000):
    while constraint(NN, train_uz).item()>0:
        grads = gconst(NN, train_uz)
        updates, opt_state = optimizer.update(grads, opt_state)
        NN = optax.apply_updates(NN, updates)
        co += 1

    grads = gobj(NN, train_uz)
    updates, opt_state = optimizer.update(grads, opt_state)
    NN = optax.apply_updates(NN, updates)

    Lloss.append(obj(NN, train_uz).item())
    Llowest.append(lowest.item())

    if  evaluation(NN, test_uz)<lowest:
        lowest = evaluation(NN, test_uz)
        NN_best = NN

    print(f"Epoch: {epoch}, loss:{obj(NN, train_uz).item():.7f}, const:{constraint(NN, train_uz).item():.4f}, lowest:{lowest:.4f}, co:{co}, bug:{bug}")
    if epoch % 10000 == 0:
        np.save(f'/content/drive/MyDrive/No. 25 Inverse Problem/Gamma (PINN)/{TN}/Lloss.npy', Lloss)
        np.save(f'/content/drive/MyDrive/No. 25 Inverse Problem/Gamma (PINN)/{TN}/Llowest.npy', Llowest)
        with open(f'/content/drive/MyDrive/No. 25 Inverse Problem/Gamma (PINN)/{TN}/NN (best).pkl', 'wb') as f:
            pickle.dump(NN_best, f)
        with open(f'/content/drive/MyDrive/No. 25 Inverse Problem/Gamma (PINN)/{TN}/NN_{epoch}.pkl', 'wb') as f:
            pickle.dump(NN, f)

流式输出内容被截断，只能显示最后 5000 行内容。
Epoch: 1995000, loss:0.0000038, const:0.0000, lowest:0.0000, co:0, bug:0
Epoch: 1995001, loss:0.0000040, const:0.0000, lowest:0.0000, co:0, bug:0
Epoch: 1995002, loss:0.0000041, const:0.0000, lowest:0.0000, co:0, bug:0
Epoch: 1995003, loss:0.0000041, const:0.0000, lowest:0.0000, co:0, bug:0
Epoch: 1995004, loss:0.0000040, const:0.0000, lowest:0.0000, co:0, bug:0
Epoch: 1995005, loss:0.0000039, const:0.0000, lowest:0.0000, co:0, bug:0
Epoch: 1995006, loss:0.0000038, const:0.0000, lowest:0.0000, co:0, bug:0
Epoch: 1995007, loss:0.0000037, const:0.0000, lowest:0.0000, co:0, bug:0
Epoch: 1995008, loss:0.0000036, const:0.0000, lowest:0.0000, co:0, bug:0
Epoch: 1995009, loss:0.0000037, const:0.0000, lowest:0.0000, co:0, bug:0
Epoch: 1995010, loss:0.0000037, const:0.0000, lowest:0.0000, co:0, bug:0
Epoch: 1995011, loss:0.0000038, const:0.0000, lowest:0.0000, co:0, bug:0
Epoch: 1995012, loss:0.0000038, const:0.0000, lowest:0.0000, co:0, bug:0
Epoch: 1995013, loss:0.0